# Sequence Labeling in python

### Initialise the transition, start and emission matrix . The states stand for high and low.  The HMM model is given in the assignment itself. 

In [1]:
import numpy as np

P= np.array([[0.6, 0.4],[0.5,0.5]])

S= np.array([0.5, 0.5])

O= np.array([[0.3,0.2,0.2,0.3],[0.2,0.3,0.3,0.2]])

state={}
state[0]='L'
state[1]='H'

DNA={}
DNA['A']=0
DNA['C']=1
DNA['G']=2
DNA['T']=3


### A stupid attempt to show you why the exhaustive search is a bad, bad option for HMM modelling. 

In [47]:
from itertools import product

import time 
def exhaustive_search(sequence):
    
    M= len(sequence)
    state_len= len(S)
    
    # track the best sequence and its score
    best=(None,float('-inf'))
    
    # basically loop will run for |states|^M 
    for ss in product(range(state_len),repeat=M):
        
        score= S[ss[0]]*O[ss[0],DNA[sequence[0]]]
        
        for i in range(1,M):
            score*= P[ss[i-1],ss[i]]*O[ss[i],DNA[sequence[i]]]
            
        
        #print(','.join([state[k] for k in ss]),score)
    
        if score > best[1]:
            best= (ss,score)
    
    return best
def viterbi(sequence):
    M = len(sequence)
    state_len = len(S)
    
    T1 = np.zeros((state_len,M))
    T2 = np.zeros((state_len,M))
    for state in range(state_len):
        T1[state,0] = S[state]*O[state,DNA[sequence[0]]]
        T2[state,0] = -1
    for i in range(1,M):
        for state in range(state_len):
            temp = T1[:,i-1]*P[:,state]*O[state,DNA[sequence[i]]]
            T1[state,i] = np.max(temp)
            T2[state,i] = np.argmax(temp)
 
    seq = np.zeros(M)
    seq[M-1] = np.argmax(T1[:,M-1])
    best_score = np.max(T1[:,M-1])
    for state in range(M-2,-1,-1):
        seq[state] = T2[int(seq[state+1]),state+1]    
    print(seq)
    return seq, best_score

In [49]:
viterbi(sequences[1])

[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


(array([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), 1.3774950719999995e-09)

In [3]:
sequences=['GGC','GGCAAGATCAT','GAGAGGAGAGAGAGAGAGA']

import time
for sequence in sequences:
    
    t=time.time()
    best=exhaustive_search(sequence)
    t2=time.time()-t
    
    print('For the sequence '+ sequence+ ' of length '+ str(len(sequence))+' time taken was '+ str(round(t2,3))+'s' )
    print('The sequence '+ ','.join([state[k] for k in best[0]])+ ' gave the best score of '+ str(best[1]))
    print('\n')

For the sequence GGC of length 3 time taken was 0.0s
The sequence H,H,H gave the best score of 0.003375


For the sequence GGCAAGATCAT of length 11 time taken was 0.039s
The sequence H,H,H,L,L,L,L,L,L,L,L gave the best score of 1.3774950719999997e-09


For the sequence GAGAGGAGAGAGAGAGAGA of length 19 time taken was 4.631s
The sequence H,L,L,L,H,H,L,L,L,L,L,L,L,L,L,L,L,L,L gave the best score of 1.3326697514029538e-16




# Dataset for this assignment: Brown corpus tagged with the Universal Tagset.

## This will be your training set. The remaining 100 sentences will be used as your test data.

In [5]:
from nltk.corpus import treebank,brown

corpus = brown.tagged_sents(tagset='universal')[:-100] 

tag_dict={}
word_dict={}

for sent in corpus:
    for elem in sent:
        w = elem[0]
        tag= elem[1]

        if w not in word_dict:
            word_dict[w]=0

        if tag not in tag_dict:
            tag_dict[tag]=0

        word_dict[w]+=1
        tag_dict[tag]+=1

print(len(word_dict))
print(len(tag_dict))
        
test_data= brown.tagged_sents(tagset='universal')[-10:]

print(len(test_data))

55907
12
10


## Module to implement CRF. 

In [62]:
# pip3 install sklearn-crfsuite # install this please

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

train_sents= corpus

def word2features(sent,i):
    word = sent[i][0]
    
    features ={
    'bias': 1.0,
    'low' : len(word),
    'isFirst' : i == 0,
    'startsWithUpper' : word[0].isupper()
    }
                
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for i,label in sent]



In [63]:
X_train=[sent2features(s) for s in train_sents]
y_train=[sent2labels(s) for s in train_sents]

X_test=[sent2features(s) for s in test_data]
y_test=[sent2labels(s) for s in test_data]

print("Done")

Done


In [64]:

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [60]:
y_pred = crf.predict(X_test)
labels=list(crf.classes_)

metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

/home/ayush/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ayush/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.3713056343117201

In [56]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

          .      0.800     0.242     0.372        33
          X      0.023     1.000     0.044         3
        ADJ      0.000     0.000     0.000        18
        ADP      0.179     0.185     0.182        27
        ADV      0.000     0.000     0.000         9
       VERB      0.000     0.000     0.000        35
        DET      0.121     0.121     0.121        33
       CONJ      0.000     0.000     0.000         7
       NOUN      0.242     0.157     0.190        51
       PRON      0.000     0.000     0.000        12
        PRT      0.000     0.000     0.000        11
        NUM      0.000     0.000     0.000         0

avg / total      0.199     0.117     0.130       239



/home/ayush/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ayush/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[['DET',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.',
  'DET',
  'NOUN',
  '.'],
 ['NOUN',
  '.',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  '.',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  '.',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  '.',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  '.'],
 ['DET',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'NOUN',
  'VERB',
  'VERB',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.'],
 ['PRON',
  'VERB',
  'DET',
  'NOUN',
  'ADP',